In [ ]:
# default_exp core

# Calorie counter

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

First you will provide your height, weight and age and calculate your Basal Metabolic Rate, the number of calories required to keep your body functioning at rest.

Then after choosing your usual level of activity the Active Metabolic Rate, the number of calories that we consume on a daily basis depending on our height, gender, age, weight and entered activity level whilst maintaining current weight, will be calculated.

source for the formulas https://www.verywellfit.com/how-many-calories-do-i-need-each-day-2506873

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide

import pandas as pd
import regex as re

In [ ]:
#export
def input_body_parameters(message_dict):
    """
    returns a dictionary with user input of body parameters: height, weight, age, gender and activity level

    :param message_dict: a dictionary with body parameters as keys and message for user as values
    :type message_dict: dict

    :raises AssertError: if inputs of type int are negative
    :raises AssertError: if inputs of type string are not 'w' or 'm'
    
    :rtype: int
    :return: multiplication of all arguments 
    """
    input_values = [170, 56, 41,'m', 1]
    body_params = {}
    n = range(len(message_dict))
    zipped = zip(message_dict.keys(), n)
    assert len(input_values) == len(message_dict)
    for param, nr in zipped:
        if param =='gender':
            try:
                body_params[param] = input(message_dict[param])
            except:
                body_params[param] = input_values[nr]
            assert body_params[param] == 'w' or body_params[param] == 'm', 'gender value should be w or m'
        else:
            try:
                body_params[param] = int(input(message_dict[param]))
            except:
                body_params[param] = input_values[nr]
            assert body_params[param] > 0, 'Please provide a positive value'

        body_params[param] = body_params[param]

        print(f'Your {param} is {body_params[param]}')
    return body_params


def calculate_bmr_amr(body_parameters):
    """
    returns basal metabolic rate(bmr) and active metabolic rate (amr) 
    
    source for the formulas used for calculations: https://www.verywellfit.com/how-many-calories-do-i-need-each-day-2506873
    
    :param body_parameters 
    :type body_parameters: dict

    :return bmr
    :rtype float
    :return amr
    :rtype float
    """
    dict_activity = {1:1.2, 2:1.37, 3:1.55, 4:1.725, 6:1.9}

    if body_parameters['gender'] == 'w':
        bmr = float(655.1 + (9.563 * body_parameters['weight']) +
            (1.850 * body_parameters['height']) - 
            (4.676 * body_parameters['age']))
    else:
        bmr = float(66.47 + (13.75 * body_parameters['weight']) + 
            (5.003 * body_parameters['height']) - 
            (6.755 * body_parameters['age']))

    amr = bmr * dict_activity[body_parameters['activity']]
    return bmr, amr
    

def calculate_burned_calories(body_parameters):
    """
    Returns the amount of calories burned during activity and during the day apart of activity.
    
    To calculate calories for specific activity its MET value is chosen from the table.
    To assign MET user should type in the name of her activity for the search.
    If user doesn't provide this information, activity is considered to be standing and light effort with MET of 1.4
    Calories burned during rest of the day are calculated with standard MET of 1.2.
    :param body_parameters
    :type body_parameters: dict

    :return burned_cal
    :rtype float
    :return bmr_cal
    :rtype float
    """

    met_list = pd.read_csv('data/met_list_activities.csv',
                            sep=';', 
                            #encoding='ANSI', 
                            decimal=',', 
                            error_bad_lines=False)
    nr = 0
    met = 0 
    time = 0
    your_act = ''
    while met == 0:
        try:
            your_act = input('Type in the activity you were doing today:')
        except:
            your_act = 'standing, light effort tasks'
        # cheking, if the typed activity is in the list of activities
        if len(met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)]) > 1:
            print('\n', met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)])
            try:
                nr = int(input('\nSelect MET of your activity from the list'))
            except:
                nr = 176
            #met = float(met_list['METs'].iloc[nr])
            #activity = met_list['SPECIFIC MOTION'].iloc[nr]
        elif len(met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)]) == 0:
            print('Please type again')
        elif len(met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)]) == 1:
            nr = met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)].index
        met = float(met_list['METs'].iloc[nr])
        activity = met_list['SPECIFIC MOTION'].iloc[nr]
    try:
        time = int(input('Type in the time of your activity for today in minutes:'))
    except:
        time = 30

    burned_cal = met * body_parameters['weight'] * time/60
    bmr_cal = 1.2 * body_parameters['weight'] * (24 - time / 60)
    print(f'\nDuring {time} min of {activity} you burned {burned_cal:.0f} kcal.')
    print(f'Your total daily energy expenditure was {round((burned_cal + bmr_cal), 2):.0f} kcal today')
    return burned_cal, bmr_cal

In [ ]:
#hide
# idea! add possibility to correct initial data

message_activity_level = 'Please choose your level of activity:\
                \n1 - little to no exercise \
                \n2 - light exercise 1–3 days a week \
                \n3 - moderate exercise 3–5 days a week \
                \n4 - hard exercises 6–7 days a week \
                \n5 - physically demanding job or particularly challenging exercise routine'

message_dict = {'height': 'Input height in cm',
                'weight': 'Input weight in kg',
                'age': 'Input age in full years',
                'gender': 'Input biological gender as w or m',
                'activity': message_activity_level}

body_params = input_body_parameters(message_dict)

bmr, amr = calculate_bmr_amr(body_params)
print(f'\nYour BMR is {bmr:.1f}')
print(f'To stay at your current weight you need to consume {amr:.0f} calories')

Your height is 180
Your weight is 67
Your age is 45
Your gender is m
Your activity is 1

Your BMR is 1584.3
To stay at your current weight you need to consume 1901 calories


In [ ]:
#hide
print('Please provide your weight, height and age in the first part of the calculator')

burned_cal, bmr_cal = calculate_burned_calories(body_params)

C:\Users\AFEDOR~1\AppData\Local\Temp/ipykernel_9536/2494668240.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  burned_cal, bmr_cal = calculate_burned_calories(body_params)


Please provide your weight, height and age in the first part of the calculator

                  ACTIVITY                                    SPECIFIC MOTION  \
94    fishing and hunting                fishing from river bank and walking   
108   fishing and hunting  hunting large game, from a hunting stand, limi...   
129       home activities  polishing floors, standing, walking slowly, us...   
138       home activities  wash dishes, clearing dishes from table, walki...   
146       home activities  serving food, setting table, implied walking o...   
..                    ...                                                ...   
815  volunteer activities  walking, 3.0 mph, moderate speed, not carrying...   
816  volunteer activities  walking, 3.5 mph, brisk speed, not carrying an...   
817  volunteer activities  walking, 2.5 mph slowly and carrying objects l...   
818  volunteer activities  walking, 3.0 mph moderately and carrying objec...   
819  volunteer activities  walking, 3.5

In [ ]:
#hide
"""import pandas as pd

met_list = pd.read_csv('data/met_list_activities.csv',
                            sep=';', 
                            #encoding='ANSI', 
                            decimal=',', 
                            error_bad_lines=False)
nr = 180

met_list.head()
"""

,ACTIVITY,SPECIFIC MOTION,METs
0,bicycling,"bicycling, mountain, uphill, vigorous",14.0
1,bicycling,"bicycling, mountain, competitive, racing",16.0
2,bicycling,"bicycling, BMX",8.5
3,bicycling,"bicycling, mountain, general",8.5
4,bicycling,"bicycling, 10 mph, leisure, to work or for ple...",4.0


In [ ]:
"""
#met_list
met = 0
time = 30

print(met_list['METs'].iloc[met])

while met == 0:
    your_act = input('Type in the activity you were doing today')
    # cheking, if the typed activity is in the list of activities
    if len(met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)]) > 1:
        print('\n', met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)])
        met = int(input('\nSelect number of your activity from the list'))
    elif len(met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)]) == 1:
        met = met_list[met_list['SPECIFIC MOTION'].str.contains(your_act)].index
    else:
        print('There was no matching activity. Please try another activity name')
    print(met_list['METs'].iloc[met])
    burned_cal = met_list['METs'].iloc[met] * body_params['weight'] * time/60
    bmr_cal = 1.2 * body_params['weight'] * (24 - time / 60)
    print('\nYour Total Daily Energy Expenditure was {} kcal today.\n\
    You burned {} kcal during your activity'.format(
        round((burned_cal + bmr_cal), 2),  
        burned_cal))
"""